<a href="https://colab.research.google.com/github/illusion173/CS555UAV/blob/main/CS555Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Necessary Commands to run before import
%pip install ultralytics
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: jupyterlab_widgets
    Found existing installation: jupyterlab_widgets 3.0.13
    Uninstalling jupyterlab_widgets-3.0.13:
      Successfully uninstalled jupyterlab_widgets-3.0.13
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [13]:
# Import needed forms
import os
import collections
import pandas as pd
import numpy as np
import functools
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
import warnings
import zipfile
from google.colab import drive
import ultralytics
import shutil

**GOOGLE DRIVE MOUNTING/UNZIPPING OF DATASET**

In [3]:
# DO NOT TOUCH PLEASE, loc of your zip file should match this
# the zip should be in the root of your google drive.

GOOGLE_DRIVE_DATA_DIR_PATH = "/content/drive/"
drive.mount(GOOGLE_DRIVE_DATA_DIR_PATH, force_remount=True)
%cd /content/drive/MyDrive/
# Unzip the file into /tmp using -d, shouldn't take too long <1 min
!unzip -q "uav_datasets.zip" -d /tmp


Mounted at /content/drive/
/content/drive/MyDrive


**DATA INGESTION/ORGANIZATION**

In [6]:
# KEEP NOTE THAT THE DIRECTORIES MADE IN THIS CELL ARE IN THE /tmp dir and WILL BE DELETED
YOLO_DATASET_FOLDER = "/tmp/drone_dataset_yolo/dataset_txt/" # This dir made in the above unzip cell
TRAIN_IMAGES_FOLDER = "/tmp/train/images"
TRAIN_LABELS_FOLDER = "/tmp/train/labels"
VAL_IMAGES_FOLDER = "/tmp/val/images"
VAL_LABELS_FOLDER = "/tmp/val/labels"
os.makedirs(TRAIN_IMAGES_FOLDER, exist_ok=True)
os.makedirs(TRAIN_LABELS_FOLDER, exist_ok=True)
os.makedirs(VAL_IMAGES_FOLDER, exist_ok=True)
os.makedirs(VAL_LABELS_FOLDER, exist_ok=True)

In [10]:
# Creating necessary lists for organizing the files
images = []
labels = []

# Collect image and label file paths from the dataset folder
for file_name in os.listdir(YOLO_DATASET_FOLDER):
    if file_name.endswith((".jpg", ".png")):  # Assuming images are .jpg or .png
        base_name = os.path.splitext(file_name)[0]
        label_file = base_name + ".txt"  # Assuming labels are in YOLO format (.txt)

        # Check if the corresponding label file exists
        if os.path.exists(os.path.join(YOLO_DATASET_FOLDER, label_file)):
            images.append(file_name)
            labels.append(label_file)

In [16]:
# Split the data into training and validation sets 80% train, 20% val
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)


In [15]:
# Define helper function to organize the files into appropriate train and value folders
def move_files(file_list, src_folder, dest_folder):
    for file in file_list:
        shutil.copy(os.path.join(src_folder, file), os.path.join(dest_folder, file))

In [17]:
# Move the training images and labels to the train folder
move_files(train_images, YOLO_DATASET_FOLDER, TRAIN_IMAGES_FOLDER)
move_files(train_labels, YOLO_DATASET_FOLDER, TRAIN_LABELS_FOLDER)

# Move the validation images and labels to the val folder
move_files(val_images, YOLO_DATASET_FOLDER, VAL_IMAGES_FOLDER)
move_files(val_labels, YOLO_DATASET_FOLDER, VAL_LABELS_FOLDER)